# combine voting outcomes with imagery

In [2]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from collections import defaultdict
import random
import numpy as np
import operator
import os
import cv2
%matplotlib notebook

__Load voting counts from June 5th 2019 election__

In [3]:
dat = pd.read_csv('total_fintal.csv',sep=';',encoding='latin1')
dat = dat[dat['Resultattype'] == 'Opstillingskreds'].reset_index(drop=True)
# parse name of opstillingskrese
dat['Sted'] = dat['Sted'].str.split(' ').str[1:].str.join(' ')

__Load imagery data__

In [4]:
imagery_info = pd.read_csv('points_data_zoom16.csv',names=['file_id','opstillingsnavn','lat','lon'],header=0,
                          encoding='utf-8')
# manually fixing one error as one electoral district's name is not consistent across the shape and voting datafiles
imagery_info.loc[imagery_info['opstillingsnavn']=='Utterslev','opstillingsnavn'] = 'Bispebjerg'

__Load electoral district boundaries ("Opstillingskreds" in danish)__

In [5]:
with open('opstillingskreds.geojson','rb') as f:
    shapes = json.load(f)

In [6]:
parties = [u'A. Socialdemokratiet',u'B. Radikale Venstre',u'O. Dansk Folkeparti',
           u'V. Venstre, Danmarks Liberale Parti',u'Ø. Enhedslisten - De Rød-Grønne',
          'C. Det Konservative Folkeparti'
          ]

In [7]:
ordered_ = defaultdict(list)
# go through electoral districts
for index, row in dat.iterrows():
    for p in parties:
        p_votes = row[p]/float(row['Gyldige stemmer'])
        ordered_[p].append((p_votes,row['Sted']))
        
# find top 10 districts for each part
top_districts = dict()
for p in parties:
    top_districts[p] = sorted(ordered_[p],reverse=True,key=operator.itemgetter(0))[:10]

In [ ]:
# add imagery info for each district in top districts
tiles = dict()
for p in top_districts:
    tiles[p] = []
    for _,d in top_districts[p]:
        tiles[p].extend(imagery_info[imagery_info['opstillingsnavn'] == d]['file_id'])

__load images and merge tiles into one figure__

In [13]:
# load images
path = 'tiles/zoom16/'
dim = 600

for p in tiles:

    ordered_img = [] # to store data

    # go through images
    for image in random.sample(tiles[p],12):
        # open image
        img = cv2.imread(path + '%03d_600x600.png' % image)

        # add to array
        ordered_img.append(img)
        
    # horizontally stack images
    hstack_image = None
    # go through ordered images
    for img in ordered_img:

        # horizontally append image
        if hstack_image is None:
            hstack_image = img
        else:
            hstack_image = np.hstack((hstack_image,img))

    # split linear image into multiple rows
    row_length = 4
    buffer_ = 4
    hbuffer = 1*np.ones((dim,buffer_,3))
    vbuffer = 1*np.ones((buffer_,dim*row_length + (row_length+1)*buffer_,3))

    mosaic = vbuffer
    for i in range(0,hstack_image.shape[1],dim*row_length):

        # first construct row by adding buffer between images and at the ends
        row = hbuffer # start with buffer
        for j in range(0,dim*row_length,dim):
            row = np.hstack((row,hstack_image[:,i+j:i+j+dim,:])) # add image
            row = np.hstack((row,hbuffer)) # add buffer

        mosaic = np.vstack((mosaic,row)) # add row
        mosaic = np.vstack((mosaic,vbuffer)) # add buffer

    # save image
    cv2.imwrite("tiles/%s.png" % p, mosaic)